**Homework 1: Data Prospectus**

Submitted by: Uday Sharma (udayshar)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings("ignore")
plt.style.use("seaborn")

In [3]:
df = pd.read_csv('OFFERING.tsv', sep='\t')

**List of Variables**

| Number | Variable | Description | Type |
| --- | --- | --- | --- 
| 1 | ACCESSIONNUMBER | This is the unique identifier for each row (company), and it corresponds to an EDGAR submission. It is alphanumeric with 20 characters, and of the format *xxxxxxxxxx-xx-xxxxxx*. | string |
| 2 | INDUSTRYGROUPTYPE | This describes the type of industry the company is from. | category |
| 3 | INVESTMENTFUNDTYPE | This describes the type of security offered by the company. | category |
| 4 | IS40ACT | This is a flag variable to indicate whether the issuer is registered as an investment company under the Investment Company Act of 1940. | category |
| 5 | REVENUERANGE | The dollar range for the revenue of the company, if disclosed. This is only applicable to companies that are not hedge or other pooled investment funds. | category |
| 6 | AGGREGATENETASSETVALUERANGE | The dollar range for the aggregate net asset value for the company, if disclosed. This is only applicable to hedge and other pooled investment funds. | category |
| 7 | FEDERALEXEMPTIONS_ITEMS_LIST | Lists the exemptions given to the company under the Securities Act. | category (multiple possible) |
| 8 | ISAMENDMENT | Whether the listing is an amendment (as opposed to a new notice). | boolean |
| 9 | PREVIOUSACCESSIONNUMBER | The previous accession number the company had, if any. | string |
| 10 | SALE_DATE | The date the security was sold. | datetime |
| 11 | YETTOOCCUR | Whether the sale is yet to occur.  | boolean |
| 12 | MORETHANONEYEAR | Whether the duration of the offering is more than one year. | boolean |
| 13 | ISEQUITYTYPE | Whether the offering is of the type equity. | boolean |
| 14 | ISDEBTTYPE | Whether the offering is of the type debt. | boolean |
| 15 | ISOPTIONTOACQUIRETYPE | Whether the offering is an option, warrant, or other right to acquire another security. | boolean |
| 16 | ISSECURITYTOBEACQUIREDTYPE | Whether the offering is to be acquired upon exercise of option, warrant or other right to acquire security. | boolean |
| 17 | ISPOOLEDINVESTMENTFUNDTYPE | Whether the offering is a pooled investment fund. | boolean |
| 18 | ISTENTANTINCOMMONTYPE | Whether the offering is a tenant-in-common security. | boolean |
| 19 | ISMINERALPROPERTYTYPE | Whether the offering is a mineral property security. | boolean |
| 20 | ISOTHERTYPE | Whether the offering is of some other type. | boolean |
| 21 | DESCRIPTIONOFOTHERTYPE | If the offering is of some other type, then its description. | string |
| 22 | ISBUSINESSCOMBINATIONTRANS | Whether the offering was made in connection with a business combination transaction, like an M&A. | boolean |
| 23 | BUSCOMBCLARIFICATIONOFRESP | The clarification of response provided, if any. | string |
| 24 | MINIMUMINVESTMENTACCEPTED | Minimum investment accepted from an outside investor. | numeric |
| 25 | OVER100RECIPIENTFLAG | Whether there are over 100 recipients for the offering. | boolean |
| 26 | TOTALOFFERINGAMOUNT | Total amount that's being offered. | numeric |
| 27 | TOTALAMOUNTSOLD | Total amount that was sold from the amount offered. | numeric |
| 28 | TOTALREMAINING | Total amount remaining to be sold. | numeric |
| 29 | SALESAMTCLARIFICATIONOFRESP | The clarification of response provided for the sales amount, if any. | string |
| 30 | HASNONACCREDITEDINVESTORS | Whether the securities in the offering have been or may be sold to persons who do not qualify as accredited investors. | boolean |
| 31 | NUMBERNONACCREDITEDINVESTORS | Number of such non-accredited investors, if any. | numeric |
| 32 | TOTALNUMBERALREADYINVESTED | Total number of all investors that have invested in the offering, regardless of their accredited status.| numeric |
| 33 | SALESCOMM_DOLLARAMOUNT | US dollar amount of sales commissions.| numeric |
| 34 | SALESCOMM_ISESTIMATE | Whether the sales commission amount is an estimate.| boolean |
| 35 | FINDERSFEE_DOLLARAMOUNT | The finders fee in USD.| numeric |
| 36 | FINDERSFEE_ISESTIMATE | Whether the finders fee is an estimated value. | boolean |
| 37 | FINDERFEECLARIFICATIONOFRESP | The clarification of response provided for the finders fee, if any. | string |
| 38 | GROSSPROCEEDSUSED_DOLLARAMOUNT | The dollar amount of the gross proceeds of the offering used (or planned to be used) to make payments to promoters, executive officers or directors.| numeric |
| 39 | GROSSPROCEEDSUSED_ISESTIMATE | Whether the dollar amount for the gross proceeds above is an estimate.| boolean |
| 40 | GROSSPROCEEDSUSED_CLAROFRESP | The clarification of response provided for the above gross proceeds, if any.| string |
| 41 | AUTHORIZEDREPRESENTATIVE | Whether the entity signing the form is a duly autorized representative of the issuer.| category |

**Consistency Checks**

1\. If an offering doesn't have any non-accredited investors, either those who have already invested or those who may invest, i.e., HASNONACCREDITEDINVESTORS is False, then the field NUMBERNONACCREDITEDINVESTORS must be 0.

Note that the converse doesn't necessarily have to be true, since HASNONACCREDITEDINVESTORS considers investors whom the security may be sold to at some point in the future, even if the transaction hasn't occurred yet. However, the field NUMBERNONACCREDITEDINVESTORS only includes those non-accredited investors who have already invested. So, if NUMBERNONACCREDITEDINVESTORS is 0, it *doesn't always* imply that HASNONACCREDITEDINVESTORS must be False.

We find a lack of internal consistency here. The same value of NaN does not mean the same thing within the variable NUMBERNONACCREDITEDINVESTORS: if we know that HASNONACCREDITEDINVESTORS is False, then NUMBERNONACCREDITEDINVESTORS must be 0. So, in this case, the NaNs refer to a value of 0. However, when HASNONACCREDITEDINVESTORS is True, an NaN value for NUMBERNONACCREDITEDINVESTORS means that we *do not know* how many non-accredited investors have invested yet, since we also observe values of 0.

To fix this, for cases when HASNONACCREDITEDINVESTORS is False, we fill in zeros for the NaN values in NUMBERNONACCREDITEDINVESTORS.

In [8]:
df.loc[df.HASNONACCREDITEDINVESTORS == False,'NUMBERNONACCREDITEDINVESTORS']\
     =df[df.HASNONACCREDITEDINVESTORS == False].\
        NUMBERNONACCREDITEDINVESTORS.fillna(0)

2\. The total number of investors that have already invested must be greater than or equal to the number of non-accredited investors that have already invested, i.e., TOTALNUMBERALREADYINVESTED >= NUMBERNONACCREDITEDINVESTORS. We see if there are any rows that do not satisfy this condition.

There doesn't seem to be any particular pattern that could have caused this, since these columns do not indicate any information that could have led to the anomaly. However, at the very least, to make the data "less incorrect", we can replace TOTALNUMBERALREADYINVESTED in this case with the NUMBERNONACCREDITEDINVESTORS, as that is the minimum value for the total count.

In [11]:
df.loc[df.TOTALNUMBERALREADYINVESTED < df.NUMBERNONACCREDITEDINVESTORS,\
     'TOTALNUMBERALREADYINVESTED'] = \
     df.loc[df.TOTALNUMBERALREADYINVESTED < \
        df.NUMBERNONACCREDITEDINVESTORS, 'NUMBERNONACCREDITEDINVESTORS']

**Missing Data**

The following columns do not have any missing data:

- ACCESSIONNUMBER
- INDUSTRYGROUPTYPE
- FEDERALEXEMPTIONS_ITEMS_LIST
- ISAMENDMENT
- MORETHANONEYEAR
- ISBUSINESSCOMBINATIONTRANS
- MINIMUMINVESTMENTACCEPTED
- TOTALOFFERINGAMOUNT
- TOTALAMOUNTSOLD
- TOTALREMAINING
- HASNONACCREDITEDINVESTORS
- TOTALNUMBERALREADYINVESTED
- SALESCOMM_DOLLARAMOUNT
- FINDERSFEE_DOLLARAMOUNT
- GROSSPROCEEDSUSED_DOLLARAMOUNT

Now, we look at the columns that do have missing values. In tackling these missing values, we use the following philosophy: we make a distinction between not knowing something versus knowing that something is not applicable. NaNs should represent "lack of data", or cases where the actual value is unknown. If something is not applicable, then we fill it in with "Not Applicable" instead, since the fact that something is not applicable is additional information in itself, and should be included in the analysis.

1\. IS40ACT

In [13]:
df.IS40ACT.fillna('NaN').value_counts()

False    8226
NaN      5932
True       29
Name: IS40ACT, dtype: int64

We see that in addition to True and False values, there are also NaN values. We can distinguish between the NaNs and False values after looking at the actual Form D: the True and False values are only applicable to firms that come under the "Other Investment Fund" subsubcategory in "Banking and Financial Services" -> "Pooled Investment Fund". 

Hence, it is a better idea to change this variable from boolean to categorical, and have "Not Applicable" as the third category.

In [14]:
df.IS40ACT = df.IS40ACT.fillna('Not Applicable')

2\. INVESTMENTFUNDTYPE

In [15]:
df[df.INVESTMENTFUNDTYPE.isna()]

,ACCESSIONNUMBER,INDUSTRYGROUPTYPE,INVESTMENTFUNDTYPE,IS40ACT,REVENUERANGE,AGGREGATENETASSETVALUERANGE,FEDERALEXEMPTIONS_ITEMS_LIST,ISAMENDMENT,PREVIOUSACCESSIONNUMBER,SALE_DATE,...,TOTALNUMBERALREADYINVESTED,SALESCOMM_DOLLARAMOUNT,SALESCOMM_ISESTIMATE,FINDERSFEE_DOLLARAMOUNT,FINDERSFEE_ISESTIMATE,FINDERFEECLARIFICATIONOFRESP,GROSSPROCEEDSUSED_DOLLARAMOUNT,GROSSPROCEEDSUSED_ISESTIMATE,GROSSPROCEEDSUSED_CLAROFRESP,AUTHORIZEDREPRESENTATIVE
4,0001948612-22-000001,Other Technology,NaN,Not Applicable,No Revenues,NaN,06b,False,NaN,NaN,...,0,0,NaN,0,NaN,NaN,0,NaN,NaN,False
7,0001949037-22-000002,Other Technology,NaN,Not Applicable,No Revenues,NaN,06c,False,NaN,2022-09-05,...,8,0,NaN,0,NaN,NaN,0,NaN,NaN,False
8,0001948952-22-000001,Other Technology,NaN,Not Applicable,Decline to Disclose,NaN,06b,False,NaN,2022-09-15,...,20,0,NaN,0,NaN,NaN,0,NaN,NaN,False
13,0001946549-22-000002,Other,NaN,Not Applicable,Decline to Disclose,NaN,06b,False,NaN,2022-09-22,...,16,0,NaN,0,NaN,NaN,0,True,NaN,False
18,0001949065-22-000002,Other Technology,NaN,Not Applicable,Decline to Disclose,NaN,06c,False,NaN,2022-09-19,...,6,0,NaN,0,NaN,NaN,0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14169,0001626408-22-000001,Other,NaN,Not Applicable,Decline to Disclose,NaN,06b,True,0001626408-20-000001,2020-06-08,...,37,0,NaN,0,NaN,NaN,0,NaN,NaN,False
14177,0001876459-22-000004,Other,NaN,Not Applicable,"$1 - $1,000,000",NaN,06b,False,NaN,2022-06-15,...,1,0,NaN,0,NaN,NaN,0,NaN,NaN,False
14180,0001812620-22-000002,Other,NaN,Not Applicable,Decline to Disclose,NaN,06b,False,NaN,2022-06-15,...,12,0,NaN,0,NaN,NaN,0,NaN,NaN,False
14181,0001936135-22-000001,Residential,NaN,Not Applicable,Decline to Disclose,NaN,06b,False,NaN,2022-06-29,...,43,0,NaN,0,NaN,NaN,492000,NaN,The Managers shall earn an acquisition due dil...,False


In [16]:
df[df.INVESTMENTFUNDTYPE.isna()].IS40ACT.value_counts(), \
    df.IS40ACT.value_counts()

(Not Applicable    5932
 Name: IS40ACT, dtype: int64,
 False             8226
 Not Applicable    5932
 True                29
 Name: IS40ACT, dtype: int64)

There are around 5932 rows with missing values. On further inspection, we see that all of these missing values correspond to rows where the Investment Company Act of 1940 is not applicable. This makes sense since these companies are not Investment Funds in the first place, and hence cannot have a type.

To fix this, we replace the NaN values with another category: "Not Applicable".

In [17]:
df.INVESTMENTFUNDTYPE = df.INVESTMENTFUNDTYPE.fillna("Not Applicable")

3 and 4. REVENUERANGE and AGGREGATENETASSETVALUERANGE

In [18]:
df.REVENUERANGE.isna().sum()+df.AGGREGATENETASSETVALUERANGE.isna().sum()

14187

We observe that the sum of the number of missing values in the two columns is equal to the total number of rows we have. Looking at Form D, we see that REVENUERANGE is not applicable to firms that are "hedge" or "other investment" funds, and AGGREGATENETASSETVALUERANGE is not applicable to all the other firms.

To fix this, we set the missing values to the category "Not Applicable" instead.

In [19]:
df.REVENUERANGE = df.REVENUERANGE.fillna("Not Applicable")
df.AGGREGATENETASSETVALUERANGE = df.AGGREGATENETASSETVALUERANGE.\
    fillna("Not Applicable")

5\. PREVIOUSACCESSIONNUMBER

The missing values correspond to cases where there isn't a previous accession number. We can set the NaN values to "Not Applicable" instead.

In [20]:
df.PREVIOUSACCESSIONNUMBER = df.PREVIOUSACCESSIONNUMBER.\
    fillna("Not Applicable")

6 and 7. SALE_DATE and YETTOOCCUR

In [21]:
df.SALE_DATE.isna().sum()+df.YETTOOCCUR.isna().sum()

14187

In [22]:
df.YETTOOCCUR.fillna("NaN").value_counts()

NaN     11837
True     2350
Name: YETTOOCCUR, dtype: int64

Here, it seems like NaN for YETTOOCCUR represent the value False. So, we replace the NaNs with False. 

We also observe that the missing values are complementary: for cases when YETTOOCCUR is True, there is no associated SALE_DATE, as expected. However, fixing this is tricky: SALE_DATE is of type datetime. We should not leave values as NaN as we know the pattern, but we cannot fill in "Not Applicable" given the datatype constraint. Hence, we fill in the date "2099-12-31" instead. For any analysis on this data set, we can consider taking the product of the variables SALE_DATE and the complement of YETTOOCCUR, so that the SALE_DATE for transactions yet to occur is mulitplied by 0.

In [23]:
df.YETTOOCCUR = df.YETTOOCCUR.fillna(False)
df.SALE_DATE = df.SALE_DATE.fillna("2099-12-31")

8 to 16. ISEQUITYTYPE, ISDEBTTYPE, ISOPTIONTOACQUIRETYPE, ISSECURITYTOBEACQUIREDTYPE, ISPOOLEDINVESTMENTFUNDTYPE, ISTENANTINCOMMONTYPE, ISMINERALPROPERTYTYPE, ISOTHERTYPE, and DESCRIPTIONOFOTHERTYPE

In [24]:
df.ISEQUITYTYPE.unique(), df.ISDEBTTYPE.unique(), df.ISOPTIONTOACQUIRETYPE\
    .unique(), df.ISSECURITYTOBEACQUIREDTYPE.unique(), \
    df.ISPOOLEDINVESTMENTFUNDTYPE.unique(), \
    df.ISTENANTINCOMMONTYPE.unique(), df.ISMINERALPROPERTYTYPE.unique(), \
    df.ISOTHERTYPE.unique()

(array([True, nan], dtype=object),
 array([nan, True], dtype=object),
 array([nan, True], dtype=object),
 array([nan, True], dtype=object),
 array([True, nan], dtype=object),
 array([nan, True], dtype=object),
 array([nan, True], dtype=object),
 array([nan, True], dtype=object))

For the first 8 variables, we see that the missing values correspond to a value of False instead, i.e., when the security is not the relevant type. We can fix this by filling in False.

In [25]:
df.ISEQUITYTYPE = df.ISEQUITYTYPE.fillna(False)
df.ISDEBTTYPE = df.ISDEBTTYPE.fillna(False)
df.ISOPTIONTOACQUIRETYPE = df.ISOPTIONTOACQUIRETYPE.fillna(False)
df.ISSECURITYTOBEACQUIREDTYPE = df.ISSECURITYTOBEACQUIREDTYPE.fillna(False)
df.ISPOOLEDINVESTMENTFUNDTYPE = df.ISPOOLEDINVESTMENTFUNDTYPE.fillna(False)
df.ISTENANTINCOMMONTYPE = df.ISTENANTINCOMMONTYPE.fillna(False)
df.ISMINERALPROPERTYTYPE = df.ISMINERALPROPERTYTYPE.fillna(False)
df.ISOTHERTYPE = df.ISOTHERTYPE.fillna(False)

In [26]:
((df.ISOTHERTYPE == False) == (~df.DESCRIPTIONOFOTHERTYPE.isna())).sum()

0

We also see that the description for "other" type is only NaN when the security is not of "other" type. So, we can fill in "Not Applicable" for the missing values.

In [27]:
df.DESCRIPTIONOFOTHERTYPE = df.DESCRIPTIONOFOTHERTYPE.\
    fillna('Not Applicable')

17 to 20. BUSCOMBCLARIFICATIONOFRESP, SALESAMTCLARIFICATIONOFRESP, FINDERFEECLARIFICATIONOFRESP, and GROSSPROCEEDSUSED_CLAROFRESP

All of these variables are NaN when there is no provided clarification response. As the clarification is optional, we can set the NaN values to "Not Applicable".

In [28]:
df.BUSCOMBCLARIFICATIONOFRESP = df.BUSCOMBCLARIFICATIONOFRESP.\
    fillna("Not Applicable")
df.SALESAMTCLARIFICATIONOFRESP = df.SALESAMTCLARIFICATIONOFRESP.\
    fillna("Not Applicable")
df.FINDERFEECLARIFICATIONOFRESP = df.FINDERFEECLARIFICATIONOFRESP.\
    fillna("Not Applicable")
df.GROSSPROCEEDSUSED_CLAROFRESP = df.GROSSPROCEEDSUSED_CLAROFRESP.\
    fillna("Not Applicable")

21\. OVER100RECIPIENTFLAG

In [29]:
df.OVER100RECIPIENTFLAG.isna().sum()

14187

We see that all the rows have missing values for this variable. The cause here is ambigious: one possibility is that the variable is False for all the rows. Another possibility that the data from the form were not compiled – on the form, there is an option to identify additional recipients by attaching more pages, and it is possible that these pages were not counted while making the dataset.

As having all rows as "False" doesn't add any value to any analysis, to be conservative, we drop this column to avoid falsely labelling rows that are potentially "True" as "False".

In [30]:
df = df.drop(columns=['OVER100RECIPIENTFLAG'])

22\. NUMBERNONACCREDITEDINVESTORS

As discussed earlier in the consistency check, we see that the NaN values remaining refer to values that are not known, as the number of investors could be either 0 or more. So, the NaNs represent "missingness" and should be left as such.

23 to 25. SALESCOMM_ISESTIMATE, FINDERSFEE_ISESTIMATE, and GROSSPROCEEDSUSED_ISESTIMATE

In [31]:
df.SALESCOMM_ISESTIMATE.unique(), df.FINDERSFEE_ISESTIMATE.unique(), \
    df.GROSSPROCEEDSUSED_ISESTIMATE.unique()

(array([nan, True], dtype=object),
 array([nan, True], dtype=object),
 array([nan, True], dtype=object))

Here, the missing values refer to cases where the value is False, meaning that the respective dollar values are exact and not an estimate. From the actual form, we observe that these variables are represented by checkboxes that are optional to fill, i.e., they are marked when True but left empty when False. We fix them by filling in False instead.

In [32]:
df.SALESCOMM_ISESTIMATE = df.SALESCOMM_ISESTIMATE.fillna(False)
df.FINDERSFEE_ISESTIMATE = df.FINDERSFEE_ISESTIMATE.fillna(False)
df.GROSSPROCEEDSUSED_ISESTIMATE = df.GROSSPROCEEDSUSED_ISESTIMATE.\
    fillna(False)

26\. AUTHORIZEDREPRESENTATIVE

In [33]:
df.AUTHORIZEDREPRESENTATIVE.unique()

array([False, nan, True], dtype=object)

We note that there are missing values in addition to True and False. However, SEC guidelines (found [here](https://www.sec.gov/corpfin/form-d-filing)) say that all Forms D must be signed by an authorized representative. This makes the meaning of False and NaN values ambiguous, as the signature of an authorized representative is mandatory in the first place. However, we try to look for any systematic similarities for cases where the value is missing by looking for columns that have the same values for all rows.

In [34]:
same_cols = []
for column in df.columns:
    if df[df.AUTHORIZEDREPRESENTATIVE.isna()]\
        [column].unique().shape[0] == 1:
        if df[df.AUTHORIZEDREPRESENTATIVE.isna()][column]\
            .isna().sum() == 0:
            same_cols.append(column)
df[df.AUTHORIZEDREPRESENTATIVE.isna()][same_cols]

,ISSECURITYTOBEACQUIREDTYPE,ISTENANTINCOMMONTYPE,HASNONACCREDITEDINVESTORS,NUMBERNONACCREDITEDINVESTORS,FINDERSFEE_DOLLARAMOUNT,FINDERSFEE_ISESTIMATE
29,False,False,False,0.0,0,False
32,False,False,False,0.0,0,False
34,False,False,False,0.0,0,False
329,False,False,False,0.0,0,False
331,False,False,False,0.0,0,False
...,...,...,...,...,...,...
14061,False,False,False,0.0,0,False
14063,False,False,False,0.0,0,False
14078,False,False,False,0.0,0,False
14088,False,False,False,0.0,0,False


None of these columns seem to indicate why the value for AUTHORIZEDREPRESENTATIVE could be missing. So, we leave the NaN values as-is.

**Translating the Variables to an Appropriate Format**

First, we convert all the numeric values to the appropriate format. The columns *TOTALOFFERINGAMOUNT* and *TOTALREMAINING* contain some values marked 'Indefinite', which we replace with infinity. According to which data type is appropriate, some columns are converted to integers, and others to floats (when decimal points make sense, or if we have missing values).

In [35]:
df.MINIMUMINVESTMENTACCEPTED = df.MINIMUMINVESTMENTACCEPTED.\
    astype('float64')
df.TOTALOFFERINGAMOUNT = df.TOTALOFFERINGAMOUNT.\
    replace('Indefinite',np.inf).astype('float64')
df.TOTALAMOUNTSOLD = df.TOTALAMOUNTSOLD.astype('float64')
df.TOTALREMAINING = df.TOTALREMAINING.replace('Indefinite',\
    np.inf).astype('float64')
df.NUMBERNONACCREDITEDINVESTORS = \
    df.NUMBERNONACCREDITEDINVESTORS.astype('float64')
df.TOTALNUMBERALREADYINVESTED = df.TOTALNUMBERALREADYINVESTED\
    .astype('int64')
df.SALESCOMM_DOLLARAMOUNT = df.SALESCOMM_DOLLARAMOUNT\
    .astype('float64')
df.FINDERSFEE_DOLLARAMOUNT = df.FINDERSFEE_DOLLARAMOUNT\
    .astype('float64')
df.GROSSPROCEEDSUSED_DOLLARAMOUNT = \
    df.GROSSPROCEEDSUSED_DOLLARAMOUNT.astype('float64')

Next, we convert the variables that are categorical, and lack a natural ordering. For the special case of categorical variables where we only have True or False values (and no missing values), we can convert them to the type boolean.

In [36]:
df.ISAMENDMENT = df.ISAMENDMENT.astype(bool)
df.YETTOOCCUR = df.YETTOOCCUR.astype(bool)
df.MORETHANONEYEAR = df.MORETHANONEYEAR.astype(bool)
df.ISEQUITYTYPE = df.ISEQUITYTYPE.astype(bool)
df.ISDEBTTYPE = df.ISDEBTTYPE.astype(bool)
df.ISOPTIONTOACQUIRETYPE = df.ISOPTIONTOACQUIRETYPE.astype(bool)
df.ISSECURITYTOBEACQUIREDTYPE = df.ISSECURITYTOBEACQUIREDTYPE.astype(bool)
df.ISPOOLEDINVESTMENTFUNDTYPE = df.ISPOOLEDINVESTMENTFUNDTYPE.astype(bool)
df.ISTENANTINCOMMONTYPE = df.ISTENANTINCOMMONTYPE.astype(bool)
df.ISMINERALPROPERTYTYPE = df.ISMINERALPROPERTYTYPE.astype(bool)
df.ISOTHERTYPE = df.ISOTHERTYPE.astype(bool)
df.ISBUSINESSCOMBINATIONTRANS = df.ISBUSINESSCOMBINATIONTRANS.astype(bool)
df.HASNONACCREDITEDINVESTORS = df.HASNONACCREDITEDINVESTORS.astype(bool)
df.SALESCOMM_ISESTIMATE = df.SALESCOMM_ISESTIMATE.astype(bool)
df.FINDERSFEE_ISESTIMATE = df.FINDERSFEE_ISESTIMATE.astype(bool)
df.GROSSPROCEEDSUSED_ISESTIMATE = df.GROSSPROCEEDSUSED_ISESTIMATE\
    .astype(bool)

df.INDUSTRYGROUPTYPE = df.INDUSTRYGROUPTYPE.astype('category')
df.INVESTMENTFUNDTYPE = df.INVESTMENTFUNDTYPE.astype('category')
df.IS40ACT = df.IS40ACT.astype('category')
df.REVENUERANGE = df.REVENUERANGE.astype('category')
df.AGGREGATENETASSETVALUERANGE = df.AGGREGATENETASSETVALUERANGE\
    .astype('category')
df.AUTHORIZEDREPRESENTATIVE = df.AUTHORIZEDREPRESENTATIVE.astype('category')

For the variable FEDERALEXEMPTIONS_ITEMS_LIST, we see that it is a list of different exemptions. In order to capture the fact that some rows might have exemptions in common but not have an identical list, we split it into different columns for each exemption, where each column is a boolean.

In [37]:
exemptions = set()
for exemp_list in df.FEDERALEXEMPTIONS_ITEMS_LIST.unique():
    empts = exemp_list.split(", ")
    exemptions.update(empts)

def create_col(exemp_list, col_name):
    empts = exemp_list.split(", ")
    if col_name in empts:
        return True
    else:
        return False

for col in exemptions:
    df.loc[:, col] = df.FEDERALEXEMPTIONS_ITEMS_LIST.apply(lambda x: \
        create_col(x, col))
    df.loc[:, col] = df.loc[:, col].astype(bool)

Finally, we set SALE_DATE to type datetime.

In [38]:
df.SALE_DATE = df.SALE_DATE.astype('datetime64')